In [1]:
# pip install frozendict 
# pip install pyarrow
# pip install -e ./streamline_package
# pip install ray[default]

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from streamline.delayed import delay_lib, step, Delayed, eval_delay
from streamline import RunEnv, load_runenv, Pipeline, Function, Var, DebugPipeline
from streamline.utilities import mainify

In [4]:
from typing import *
import pandas as pd

In [5]:
# import ray

# ray.init()

In [6]:
pipe2 = Pipeline([Function(lambda a, b, c: (a, b, c), args=[Var('s')], kw={'b': Var('t'), 'c': 4})]).add_delete([Var('m')])
pipe2.run({'s': 2, 't': 3, 'm': 5})

{'s': 2, 't': 3, '_': (2, 3, 4), '__preamble__': {'__steps__': [{'step': Function(arg_cat='', tags={'function'}), 'kwargs': {}}, {'step': Delete(arg_cat='', tags={'delete'}), 'kwargs': {}}]}}

In [7]:
pipe3 = pipe2.copy()
pipe3.rename({'s': 'n', 't': 'a', 'm': 'd'})
pipe3.run({'n': 2, 'a': 3, 'd': 4})

{'n': 2, 'a': 3, '_': (2, 3, 4), '__preamble__': {'__steps__': [{'step': Function(arg_cat='', tags={'function'}), 'kwargs': {}}, {'step': Delete(arg_cat='', tags={'delete'}), 'kwargs': {}}]}}

In [8]:
isinstance(OrderedDict(), dict)

True

In [9]:
renv = RunEnv({
    'a': 3,
    'df': pd.DataFrame([[4]])
})

renv.save(
    './ex/'
)
# display(renv._env)
renv2 = load_runenv('./ex/')

In [10]:
import numpy as np
import time

def dummy():
    print(id(time))
    time.sleep(1)
    return 
    
def dummy2(env, kw):
    env['myid'] = id(env)
    env['lst'].append(id(env))

pipe = Pipeline([
    Function(lambda : 3, arg_cat='s')
]).add_step(
    step=Function(
        fun=lambda x, s, e, f, h: (x**2+1, s, e, f, h),
        args=Var('a'),
        kw={'e': Var('a'), 'f': 8, 'h': 5},
        out_var=('b', 'c', 'e', 'f', 'h'),
        arg_cat='f',
    )
).add_import_lib(
    {'numpy': 'np', 'pandas': 'pd'},    index=0
).add_step(Function(dummy, out_var='r')).add_delete(
    ['f', 'h', 'a']  # remove variables 'f' and 'h'
).add_variables_dict({
    'j': '3'
})
pipe
env = pipe.run(RunEnv({'a': 3, 'e': 'k', 'lst': []}), {'f_s': 'f_d', 'f_e': 'f_n', 'f_h': 3})
env._env

2300262129056


{'e': 'f_n',
 'lst': [],
 'np': <module 'numpy' from 'C:\\Users\\Nagato\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\numpy\\__init__.py'>,
 'pd': <module 'pandas' from 'C:\\Users\\Nagato\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\pandas\\__init__.py'>,
 '__preamble__': {'__steps__': [{'step': Function(arg_cat='', tags={'import_lib', 'function'}),
    'kwargs': {}},
   {'step': Function(arg_cat='s', tags={'function'}), 'kwargs': {}},
   {'step': Function(arg_cat='f', tags={'function'}),
    'kwargs': {'s': 'f_d', 'e': 'f_n', 'h': 3}},
   {'step': Function(arg_cat='', tags={'function'}), 'kwargs': {}},
   {'step': Delete(arg_cat='', tags={'delete'}), 'kwargs': {}},
   {'step': Function(arg_cat='', tags={'add_variables'}), 'kwargs': {}}]},
 '_': 3,
 'b': 10,
 'c': 'f_d',
 'r': None,
 'j': '3'}

In [11]:
debug_pipe = DebugPipeline(pipe)
debug_pipe.loc[: 2]

Pipeline(steps=[
  Function(arg_cat='', tags={'import_lib', 'function'}), 
  Function(arg_cat='s', tags={'function'})
], )

In [11]:
pipe.get_env_info()

[{'required_vars': set(),
  'unresolved_vars': set(),
  'unresolved_vars_accum': set(),
  'env_vars': set(),
  'removed_vars': set(),
  'added_vars': {'np', 'pd'},
  'steptype': 'Function',
  'arg_cat': '',
  'tags': {'function', 'import_lib'}},
 {'required_vars': set(),
  'unresolved_vars': set(),
  'unresolved_vars_accum': set(),
  'env_vars': {'np', 'pd'},
  'removed_vars': set(),
  'added_vars': set(),
  'steptype': 'Function',
  'arg_cat': 's',
  'tags': {'function'}},
 {'required_vars': {'a'},
  'unresolved_vars': {'a'},
  'unresolved_vars_accum': {'a'},
  'env_vars': {'np', 'pd'},
  'removed_vars': set(),
  'added_vars': {'b', 'c', 'e', 'f', 'h'},
  'steptype': 'Function',
  'arg_cat': 'f',
  'tags': {'function'}},
 {'required_vars': set(),
  'unresolved_vars': set(),
  'unresolved_vars_accum': {'a'},
  'env_vars': {'b', 'c', 'e', 'f', 'h', 'np', 'pd'},
  'removed_vars': set(),
  'added_vars': {'r'},
  'steptype': 'Function',
  'arg_cat': '',
  'tags': {'function'}},
 {'required

In [12]:
pipe4 = pipe.copy()
pipe4.rename(lambda x: 'pipe4_'+x)
pipe4.get_env_info()

[{'required_vars': set(),
  'unresolved_vars': set(),
  'unresolved_vars_accum': set(),
  'env_vars': set(),
  'removed_vars': set(),
  'added_vars': {'pipe4_np', 'pipe4_pd'},
  'steptype': 'Function',
  'arg_cat': '',
  'tags': {'function', 'import_lib'}},
 {'required_vars': set(),
  'unresolved_vars': set(),
  'unresolved_vars_accum': set(),
  'env_vars': {'pipe4_np', 'pipe4_pd'},
  'removed_vars': set(),
  'added_vars': set(),
  'steptype': 'Function',
  'arg_cat': 's',
  'tags': {'function'}},
 {'required_vars': {'pipe4_a'},
  'unresolved_vars': {'pipe4_a'},
  'unresolved_vars_accum': {'pipe4_a'},
  'env_vars': {'pipe4_np', 'pipe4_pd'},
  'removed_vars': set(),
  'added_vars': {'pipe4_b', 'pipe4_c', 'pipe4_e', 'pipe4_f', 'pipe4_h'},
  'steptype': 'Function',
  'arg_cat': 'f',
  'tags': {'function'}},
 {'required_vars': set(),
  'unresolved_vars': set(),
  'unresolved_vars_accum': {'pipe4_a'},
  'env_vars': {'pipe4_b',
   'pipe4_c',
   'pipe4_e',
   'pipe4_f',
   'pipe4_h',
   'pipe

In [13]:
pipe4.loc[:2].rename(lambda x: 'ss_'+x)

Pipeline(steps=[
  Function(arg_cat='', tags={'function', 'import_lib'}), 
  Function(arg_cat='s', tags={'function'})
], )

In [14]:
pipe4.get_env_info()

[{'required_vars': set(),
  'unresolved_vars': set(),
  'unresolved_vars_accum': set(),
  'env_vars': set(),
  'removed_vars': set(),
  'added_vars': {'ss_pipe4_np', 'ss_pipe4_pd'},
  'steptype': 'Function',
  'arg_cat': '',
  'tags': {'function', 'import_lib'}},
 {'required_vars': set(),
  'unresolved_vars': set(),
  'unresolved_vars_accum': set(),
  'env_vars': {'ss_pipe4_np', 'ss_pipe4_pd'},
  'removed_vars': set(),
  'added_vars': set(),
  'steptype': 'Function',
  'arg_cat': 's',
  'tags': {'function'}},
 {'required_vars': {'pipe4_a'},
  'unresolved_vars': {'pipe4_a'},
  'unresolved_vars_accum': {'pipe4_a'},
  'env_vars': {'ss_pipe4_np', 'ss_pipe4_pd'},
  'removed_vars': set(),
  'added_vars': {'pipe4_b', 'pipe4_c', 'pipe4_e', 'pipe4_f', 'pipe4_h'},
  'steptype': 'Function',
  'arg_cat': 'f',
  'tags': {'function'}},
 {'required_vars': set(),
  'unresolved_vars': set(),
  'unresolved_vars_accum': {'pipe4_a'},
  'env_vars': {'pipe4_b',
   'pipe4_c',
   'pipe4_e',
   'pipe4_f',
   '

In [15]:
isinstance(slice(None), slice)

True

In [16]:
pipe.to_list()

[Function(arg_cat='', tags={'function', 'import_lib'}),
 Function(arg_cat='s', tags={'function'}),
 Function(arg_cat='f', tags={'function'}),
 Function(arg_cat='', tags={'function'}),
 Delete(arg_cat='', tags={'delete'}),
 Function(arg_cat='', tags={'add_variables'})]

In [17]:
pipe.run(RunEnv({'a': 3, 'e': 'k', 'lst': []}), {'f_s': 'f_d', 'f_e': 'f_n', 'f_h': 3})

2187793767840


{'e': 'f_n', 'lst': [], 'np': <module 'numpy' from 'C:\\Users\\Nagato\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\numpy\\__init__.py'>, 'pd': <module 'pandas' from 'C:\\Users\\Nagato\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\pandas\\__init__.py'>, '__preamble__': {'__steps__': [{'step': Function(arg_cat='', tags={'function', 'import_lib'}), 'kwargs': {}}, {'step': Function(arg_cat='s', tags={'function'}), 'kwargs': {}}, {'step': Function(arg_cat='f', tags={'function'}), 'kwargs': {'s': 'f_d', 'e': 'f_n', 'h': 3}}, {'step': Function(arg_cat='', tags={'function'}), 'kwargs': {}}, {'step': Delete(arg_cat='', tags={'delete'}), 'kwargs': {}}, {'step': Function(arg_cat='', tags={'add_variables'}), 'kwargs': {}}]}, '_': 3, 'b': 10, 'c': 'f_d', 'r': None, 'j': '3'}

In [18]:
from streamline.pipeline import run_parallel

res_par = run_parallel([
    (pipe, {'a': 3, 'e': 'k', 'lst': []}, {'f_s': 'f_d', 'f_e': 'f_n', 'f_h': 3}),
    (pipe, {'a': 3, 'e': 'k', 'lst': []}, {'f_s': 'f_d', 'f_e': 'f_n', 'f_h': 3}),
])

In [19]:
res_par[1]

{'e': 'f_n', 'lst': [], 'np': <module 'numpy' from 'C:\\Users\\Nagato\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\numpy\\__init__.py'>, 'pd': <module 'pandas' from 'C:\\Users\\Nagato\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\pandas\\__init__.py'>, '__preamble__': {'__steps__': [{'step': Function(arg_cat='', tags={'function', 'import_lib'}), 'kwargs': {}}, {'step': Function(arg_cat='s', tags={'function'}), 'kwargs': {}}, {'step': Function(arg_cat='f', tags={'function'}), 'kwargs': {'s': 'f_d', 'e': 'f_n', 'h': 3}}, {'step': Function(arg_cat='', tags={'function'}), 'kwargs': {}}, {'step': Delete(arg_cat='', tags={'delete'}), 'kwargs': {}}, {'step': Function(arg_cat='', tags={'add_variables'}), 'kwargs': {}}]}, '_': 3, 'b': 10, 'c': 'f_d', 'r': None, 'j': '3'}

In [20]:
from streamline import eval_delay, step

eval_delay(step.arg_cat, {'step': Function(lambda x: x, arg_cat='s')})

's'

In [21]:
base_env = RunEnv({'a': 3, 'e': 'k', 'lst': []})
env_l = pipe.run_parallel([base_env], [{'f_s': 'f_d', 'f_e': 'f_n', 'f_h': i} for i in range(10)])
env_l

[[{'e': 'f_n', 'lst': [], 'np': <module 'numpy' from 'C:\\Users\\Nagato\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\numpy\\__init__.py'>, 'pd': <module 'pandas' from 'C:\\Users\\Nagato\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\pandas\\__init__.py'>, '__preamble__': {'__steps__': [{'step': Function(arg_cat='', tags={'function', 'import_lib'}), 'kwargs': {}}, {'step': Function(arg_cat='s', tags={'function'}), 'kwargs': {}}, {'step': Function(arg_cat='f', tags={'function'}), 'kwargs': {'s': 'f_d', 'e': 'f_n', 'h': 0}}, {'step': Function(arg_cat='', tags={'function'}), 'kwargs': {}}, {'step': Delete(arg_cat='', tags={'delete'}), 'kwargs': {}}, {'step': Function(arg_cat='', tags={'add_variables'}), 'kwargs': {}}]}, '_': 3, 'b': 10, 'c': 'f_d', 'r': None, 'j': '3'},
  {'e': 'f_n', 'lst': [], 'np': <module 'numpy' from 'C:\\Users\\Nagato\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\numpy\\__init__.py'>, 'pd': <module 'pandas' 

In [22]:
base_env['lst']

[]

In [23]:
[c['lst'] for c in env_l[0]]

[[], [], [], [], [], [], [], [], [], []]

In [24]:
s = list(range(10))
s = [s[i:i+5] for i in range(0, 10, 5)]
s

[[0, 1, 2, 3, 4], [5, 6, 7, 8, 9]]

In [25]:
env.gen_pipeline()

Pipeline(steps=[
  Function(arg_cat='', tags={'function', 'import_lib'}), 
  Function(arg_cat='s', tags={'function'}), 
  Function(arg_cat='f', tags={'function'}), 
  Function(arg_cat='', tags={'function'}), 
  Delete(arg_cat='', tags={'delete'}), 
  Function(arg_cat='', tags={'add_variables'})
], )

In [26]:
pipe

Pipeline(steps=[
  Function(arg_cat='', tags={'function', 'import_lib'}), 
  Function(arg_cat='s', tags={'function'}), 
  Function(arg_cat='f', tags={'function'}), 
  Function(arg_cat='', tags={'function'}), 
  Delete(arg_cat='', tags={'delete'}), 
  Function(arg_cat='', tags={'add_variables'})
], )

In [27]:
pipe.loc[[0, 1]]

Pipeline(steps=[
  Function(arg_cat='', tags={'function', 'import_lib'}), 
  Function(arg_cat='s', tags={'function'})
], )

In [28]:
pipe.loc[[0, 1]] = Function(arg_cat='d', fun=lambda x: x)

In [29]:
pipe

Pipeline(steps=[
  Function(arg_cat='d', tags={'function'}), 
  Function(arg_cat='d', tags={'function'}), 
  Function(arg_cat='f', tags={'function'}), 
  Function(arg_cat='', tags={'function'}), 
  Delete(arg_cat='', tags={'delete'}), 
  Function(arg_cat='', tags={'add_variables'})
], )

In [30]:
step.arg_cat.startswith('s')

step.arg_cat.startswith(*('s',), **{})

In [31]:
pipe.loc[step.arg_cat.startswith('d')]

Pipeline(steps=[
  Function(arg_cat='d', tags={'function'}), 
  Function(arg_cat='d', tags={'function'})
], )

In [32]:
pipe.loc[pipe.apply(lambda x: x.tags is None)]

Pipeline(steps=[
], )

In [33]:
pipe.to_list()

[Function(arg_cat='d', tags={'function'}),
 Function(arg_cat='d', tags={'function'}),
 Function(arg_cat='f', tags={'function'}),
 Function(arg_cat='', tags={'function'}),
 Delete(arg_cat='', tags={'delete'}),
 Function(arg_cat='', tags={'add_variables'})]

In [34]:
step.tags.contains(3)

(3 in step.tags)

In [35]:
step = Delayed(prefix='step')
d = delay_lib.type((step.tags.contains(3)) & (step.tags.contains(1))).__name__.isin(['bool'])
print(d)
eval_delay(d, env={'step': Function(arg_cat='s', fun=lambda x: x, tags={1, 2})})

(type(*(((3 in step.tags) & (1 in step.tags)),), **{}).__name__ in ['bool'])


True

In [36]:
eval_delay(step.tags.contains(3), env={'step': Function(arg_cat='s', fun=lambda x: x, tags={1, 2})})

False